<a href="https://colab.research.google.com/github/jblss888/cs39aa_project/blob/main/Part3_First_Attempt_Juliet_Smith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Part 3, Juliet Smith


### **Summerization of this model experimentation:**

This is a model adapting the Bert model from Assignment 5 to my dataset, LiMit, that is trying to detect motion from written sentences. 

This mnodel did improve the results from about 73% to 83% accurate. However, I'm not entirely sure if it is just due to the nature of Colab, but I was having a hard time trying to get the bits of code to run after the training was done. The training took 10-11 hours for my dataset that is roughly 23,000 observations. Because it took so long for the data to train Colab forgot the previous variables and said they were not defined. I tried rerunning some cell blocks but the when it lost the trainer variable I had to restart the training. 


The training showed that the model was the most accurate after 8000 epochs .8330, and from there it's acuracy decreased to about .8



I was looking into how to save the data more acurately, and it apears that I really don't understand all that is going on in this model. The way I have it set up is trying to compare the house prices and my sentences. 

In [ ]:
#Installations

!pip install datasets
!pip install nltk 
!pip install transformers

# https://github.com/huggingface/notebooks/blob/main/examples/token_classification.ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd # this is the tool I will use to inteact with the dataset
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split

from datasets import load_dataset # this will let me load the dataset "LiMit" from Hugging Face

import nltk # This will help me break up sentences into individual words

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Loaing "LiMiT" dataset from hugging face
dataset = load_dataset("limit")

  0%|          | 0/2 [00:00<?, ?it/s]

This redoes most of what I had done from project_1. 

Just as a reminder. This dataset is already split into training and validation datasets. I will mark them as train and val. Each of them will go through the same clean up process but I will only describe the training dataset.

In [ ]:
#This will convert the the hugging face dataset framework into a panda's dataset framework
#This dataset already has a set of test data so I don't need to set aside a portion of the training data. 

train = pd.DataFrame(dataset['train'])
val = pd.DataFrame(dataset['test'])
train.set_index('id')
val.set_index('id')

,sentence,motion,motion_entities
id,,,
0,"""We will go up stairs to my room,"" said his un...",yes,"[{'entity': 'paper', 'start_index': 79}, {'ent..."
1,Tom bent his steps in the direction of a large...,yes,"[{'entity': 'Tom', 'start_index': 0}]"
2,"They had drunk until their eyes had grown dim,...",yes,"[{'entity': 'They', 'start_index': 0}]"
3,He locked and bolted the door.,yes,"[{'entity': 'He', 'start_index': 0}, {'entity'..."
4,But appearing in Court in the guise of a mille...,yes,"[{'entity': 'Matthew Hale', 'start_index': 53}]"
...,...,...,...
995,We see a man in the pool with a cap on.,no,[]
996,A man is seen kneeling on a floor and puts his...,yes,"[{'entity': 'man', 'start_index': 2}, {'entity..."
997,the woman is in the hall and talk to the boys...,no,[]


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,  TrainingArguments, Trainer
from datasets import Dataset, load_metric

In [ ]:
MODEL_NAME = "bert-base-cased"
MAX_LENGTH=30

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, max_length=MAX_LENGTH, output_attentions=False, output_hidden_states=False)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/vocab.txt
loading file tokenize

In [ ]:
classes = train.motion.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
class_idx2tok = dict((k, v) for k, v in enumerate(classes))
print(class_tok2idx)
print(class_idx2tok)

{'yes': 0, 'no': 1}
{0: 'yes', 1: 'no'}


In [ ]:
train['label'] = train['motion'].apply(lambda x: class_tok2idx[x])
val['label'] = val['motion'].apply(lambda x: class_tok2idx[x])
train.head()

,id,sentence,motion,motion_entities,label
0,0,A little boy holding a yellow ball walks by.,yes,"[{'entity': 'little boy', 'start_index': 2}, {...",0
1,1,The camel walks as the woman leans forward.,yes,"[{'entity': 'camel', 'start_index': 4}, {'enti...",0
2,2,The man mixes up various ingredients and begi...,yes,"[{'entity': 'man', 'start_index': 4}, {'entity...",0
3,3,He continues laying tiles on the floor while ...,yes,"[{'entity': 'He', 'start_index': 0}, {'entity'...",0
4,4,Then one man stands in a field holding a woode...,yes,"[{'entity': 'man', 'start_index': 9}, {'entity...",0


In [ ]:
ds_raw = Dataset.from_pandas(train[['label','sentence']])
ds_raw[0]

{'label': 0, 'sentence': ' A little boy holding a yellow ball walks by.'}

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

ds = ds_raw.map(tokenize_function, batched=True)

  0%|          | 0/24 [00:00<?, ?ba/s]

In [ ]:
ds[0]

{'label': 0,
 'sentence': ' A little boy holding a yellow ball walks by.',
 'input_ids': [101,
  138,
  1376,
  2298,
  2355,
  170,
  3431,
  3240,
  7104,
  1118,
  119,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [ ]:
val_raw = Dataset.from_pandas(val[['label','sentence']])
val_raw[0]

{'label': 0,
 'sentence': '"We will go up stairs to my room," said his uncle, rising, and laying down the paper.'}

In [ ]:
ds_train = ds
ds_eval = val_raw.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
print(f"len(ds_train) = {len(ds_train)}")
print(f"len(ds_eval) = {len(ds_eval)}")

len(ds_train) = 23559
len(ds_eval) = 1000


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "false"

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(num_train_epochs=4, 
                                  do_train=True,
                                  report_to=None,
                                  output_dir='/content/',
                                  evaluation_strategy="steps",
                                  eval_steps=500,
                                  per_device_train_batch_size=10,
                                  per_device_eval_batch_size=10)

trainer = Trainer(model = model, 
                  args = training_args,
                  train_dataset = ds_train, 
                  eval_dataset = ds_eval,
                  compute_metrics = compute_metrics,
)

torch.set_grad_enabled(True)
trainer.train()
trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 23559
  Num Epochs = 4
  Instantaneous batch size per device = 10
  Total train batch 

Step,Training Loss,Validation Loss


keeping things active blah meh asd 

In [ ]:
df_test = val_raw
df_test.head

In [ ]:
ds_test = val_raw.map(tokenize_function, batched=True)
ds_test.head()

In [ ]:
preds = trainer.predict(test_dataset=ds_test)

In [ ]:
type(test_preds)

In [ ]:
test_preds = np.apply_along_axis(np.argmax, 1, preds.predictions) 
ds_test['preds'] = pd.Series(test_preds.tolist())

In [ ]:
df_test['preds'] = test_preds.tolist()
df_test['sentiment'] = df_test['preds'].apply(lambda x: class_idx2tok[x])
df_test.head()

In [ ]:
predictions_val = model.predict(ds_train)
disp = ConfusionMatrixDisplay(confusion_matrix(ds_eval, test_preds), display_labels=['yes', 'no'])
disp.plot()
print(f"accuracy (on X_train): {accuracy_score(ds_eval, test_preds):.4f}")